# Scraping mine data with `.apply`

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Doing this programmatically

## First, scraping a single page

### Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(chrome_options=chrome_options)

import pandas as pd
pd.set_option('display.max_colwidth', -1)

### Searching for a mine

Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm) and use Selenium to search for `3503598`

- *TIP: You might need to use the Selenium code to scroll down to the right spot on the page. Or not!*
- *TIP: Use `.send_keys` to type into the box*
- *TIP: On pages that never change, you can usually just use XPath if you're feeling lazy*

In [2]:
driver.get('https://arlweb.msha.gov/drs/drshome.htm')

In [3]:
driver.find_element_by_name('MineId').send_keys('3503598')

In [4]:
search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

### Finding reports

On the "Report Selection Page" (where you should be after you search), use Selenium to...

- Change the **Beginning Date** to `1/1/1995`
- Select the report type of `Violations`
- Click **Get Report**

.

- *TIP: Remember, if someone isn't on the page Selenium can't click it!*

In [5]:
button=driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input').click()

In [6]:
driver.find_element_by_name('BDate').send_keys('1/1/1995')

In [7]:
button=driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input').click()

### Saving reports

Save all of the rows of data on that page into a new dataframe. Each column is its own column, **and you also need to save the URL under the 'Standard' column.** Here, I even made you a blank dictionary:

```python
data = {}
data['violator'] = ''
data['contract_id'] = ''
data['citation_no'] = ''
data['case_no'] = ''
data['date_issues'] = ''
data['final_order_date'] = ''
data['section_of_act'] = ''
data['date_terminated'] = ''
data['citation'] = ''
data['s_and_s'] = ''
data['standard'] = ''
data['standard_url'] = ''
data['proposed_penalty'] = ''
data['citation_status'] = ''
data['current_penalty'] = ''
data['amount_paid'] = ''
```

- *TIP: Some of those table rows aren't what you want. How can you tell them apart from the good ones? (the previous mine owner ones are okay, I just mean the weird headers)*
- *TIP: I sense `.find_elements` + a lot of square brackets*
- *TIP: This is just like scraping a search results page!*
- *TIP: For the URL, you'll need to find the `a` inside of the cell*
- *TIP: class name is sadly not going to save your life here, because some of the `tr`s and `td`s have the same class! It's stupid. But there's a trick: CSS selectors! Something like `div#container` finds a `div` with the id of `container`, while `span.important` finds a `span` with the class of `important`. It should be helpful! And use `.find_elements_by_` + tab to see what the command is*

In [8]:
violations=[]
results=driver.find_elements_by_class_name('drsviols')
for result in results:
      
    try:
        data = {}
        
        violator=result.find_elements_by_tag_name('td')[0]
        data['violator'] = violator.text
        print(violator.text)

        contractorid=result.find_elements_by_tag_name('td')[1]
        data['contract_id'] = contractorid.text
        print(contractorid.text)

        citation_no=result.find_elements_by_tag_name('td')[2]
        data['citation_no'] = citation_no.text
        print(citation_no.text)
        
        case_no=result.find_elements_by_tag_name('td')[3]
        data['case_no'] =case_no.text
        print(case_no.text)
        
        date_issues=result.find_elements_by_tag_name('td')[4]
        data['date_issues'] = date_issues.text
        print(date_issues.text)
        
        final_order_date=result.find_elements_by_tag_name('td')[5]
        data['final_order_date'] = final_order_date.text
        print(final_order_date.text)
        
        section_of_act=result.find_elements_by_tag_name('td')[6]
        data['section_of_act'] = section_of_act.text
        print(section_of_act.text)
        
        data_terminated=result.find_elements_by_tag_name('td')[7]
        data['date_terminated'] = data_terminated.text
        print(data_terminated.text)
        
        citation=result.find_elements_by_tag_name('td')[8]
        data['citation'] = citation.text
        print(citation.text)
        
        s_and_s=result.find_elements_by_tag_name('td')[9]
        data['s_and_s'] = s_and_s.text
        print(s_and_s.text)
        
        standard=result.find_elements_by_tag_name('td')[10]
        data['standard'] = standard.text
        print(standard.text)
        
        standard_url=result.find_elements_by_tag_name('td')[10].find_element_by_tag_name('a').get_attribute('href')
        data['standard_url'] =standard_url
        
        proposed_penalty=result.find_elements_by_tag_name('td')[11]
        data['proposed_penalty'] = proposed_penalty.text

        
        citation_status=result.find_elements_by_tag_name('td')[12]
        data['citation_status'] = citation_status.text

        
        current_penalty=result.find_elements_by_tag_name('td')[13]
        data['current_penalty'] = current_penalty.text

        
        amount_paid=result.find_elements_by_tag_name('td')[14]
        data['amount_paid'] = amount_paid.text
   
        
        violations.append(data)   
        
    except:
        pass
    

 
    

Newberg Rock & Dirt
  
8992368  
000440809
5/2/2017
7/23/2017 
104(a)
5/2/2017
C
N
56.14107(a)
Newberg Rock & Dirt
  
8992368  
000440809
5/2/2017
7/23/2017 
104(a)
5/2/2017
C
N
56.14107(a)
Newberg Rock & Dirt
  
8992214  
000435717
2/7/2017
5/21/2017 
104(a)
2/8/2017
C
N
56.14107(a)
Newberg Rock & Dirt
  
8872461  
000398910
10/21/2015
1/13/2016 
104(a)
10/27/2015
C
N
56.12004
Newberg Rock & Dirt
  
8872462  
000398910
10/21/2015
1/13/2016 
104(a)
10/27/2015
C
N
56.14107(a)
Newberg Rock & Dirt
  
8790965  
000383840
4/15/2015
7/17/2015 
104(a)
4/15/2015
C
N
56.13015
Newberg Rock & Dirt
  
8790964  
000383840
4/13/2015
7/17/2015 
104(a)
4/15/2015
C
N
56.4101
Newberg Rock & Dirt
  
8790962  
000383840
4/10/2015
7/17/2015 
104(a)
4/10/2015
C
N
56.14107(a)
Newberg Rock & Dirt
  
8790963  
000383840
4/10/2015
7/17/2015 
104(a)
4/13/2015
C
Y
56.15020
Newberg Rock & Dirt
  
8694457  
000345066
1/29/2014
4/12/2014 
104(a)
1/29/2014
C
N
56.12004
Newberg Rock & Dirt
  
8694458  
000345066
1/29/

In [9]:
len(violations)

50

### Saving that data

Save the dataframe to a CSV file called `3503598-violations.csv` (that's the TDLR code)

In [10]:
df = pd.DataFrame(violations)
df.to_csv("3503598-violations.csv", index=False)

## Put that all in ONE cell that runs correctly

The **entire process**, from searching to saving as a CSV

# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [11]:
df_mines = pd.read_csv('mines-subset.csv')
df_mines.head()

,id
0,4104757
1,801306
2,3609931


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: You can zero fill if you want, but another option is that when reading in a CSV, `dtype='str'` will force everything to be a string*

In [12]:
mines_df = pd.read_csv('mines-subset.csv', dtype={'id': str})
mines_df.head()

,id
0,4104757
1,0801306
2,3609931


In [13]:
import time
import re

### Convert your one-cell scraper into a function, and use it on each row of our dataset

- *TIP: You'll be using `.apply`*
- *TIP: You won't be joining this back into your dataframe, so you don't need to `return` anything or `join` or any of that.*
- *TIP: Be careful of your **other variable names** - if you're calling the thing you're sending your function `row`, you can't use it anywhere else (like in your loop)*
- *TIP: **BE CAREFUL WHAT YOU NAME YOUR DATAFRAMES.** If you name the citations dataframe `df` it can overwrite your mine ID `df`*
- *TIP: You'll be saving a dataframe each time*
- *TIP: Be sure you change everything that refers to the mine ID to refer to the current row's ID instead of `3503598`*
- *TIP: BE SURE TO CHANGE EVERYTHING THAT REFERS TO THE MINE ID*
- *TIP: EVERYTHING, EVERYTHING, EVERYTHING! Look at the end of your function! Maybe I'm overreacting, I don't know.*
- *TIP: If you hit an error about list index out of range, see what line it's happening on and go look at the page. What's different about this page than the previous ones? (answer: the last three columns!) If you assign those columns later using `try`/`except` you should be able to get some data from those rows without throwing it all out. If you can't figure it out, just wrap it all in try/except and give up on those rows*
- *TIP: Some of the standards might not have links, either, so you might want to wrap that in a `try`/`except`, too!*

In [14]:
def get_violations(row):
    driver.get('https://arlweb.msha.gov/drs/drshome.htm')
    driver.find_element_by_name('MineId').send_keys(row['id'])
    print('sending Mines IDs')
    search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_button.click()
    print('searching')
    
    time.sleep(1)
    
    button=driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input').click()
    driver.find_element_by_name('BDate').send_keys('1/1/1995')
    button=driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input').click()
    
    time.sleep(1)
    
    all_violations=[]
    results=driver.find_elements_by_class_name('drsviols')
    for result in results:
      
        try:
            data = {}

            violator=result.find_elements_by_tag_name('td')[0]
            data['violator'] = violator.text
            print(violator.text)

            contractorid=result.find_elements_by_tag_name('td')[1]
            data['contract_id'] = contractorid.text
            print(contractorid.text)

            citation_no=result.find_elements_by_tag_name('td')[2]
            data['citation_no'] = citation_no.text
            print(citation_no.text)

            case_no=result.find_elements_by_tag_name('td')[3]
            data['case_no'] =case_no.text
            print(case_no.text)

            date_issues=result.find_elements_by_tag_name('td')[4]
            data['date_issues'] = date_issues.text
            print(date_issues.text)

            final_order_date=result.find_elements_by_tag_name('td')[5]
            data['final_order_date'] = final_order_date.text
            print(final_order_date.text)

            section_of_act=result.find_elements_by_tag_name('td')[6]
            data['section_of_act'] = section_of_act.text
            print(section_of_act.text)

            data_terminated=result.find_elements_by_tag_name('td')[7]
            data['date_terminated'] = data_terminated.text
            print(data_terminated.text)

            citation=result.find_elements_by_tag_name('td')[8]
            data['citation'] = citation.text
            print(citation.text)

            s_and_s=result.find_elements_by_tag_name('td')[9]
            data['s_and_s'] = s_and_s.text
            print(s_and_s.text)

            standard=result.find_elements_by_tag_name('td')[10]
            data['standard'] = standard.text
            print(standard.text)

            standard_url=result.find_elements_by_tag_name('td')[10].find_element_by_tag_name('a').get_attribute('href')
            data['standard_url'] =standard_url

            proposed_penalty=result.find_elements_by_tag_name('td')[11]
            data['proposed_penalty'] = proposed_penalty.text


            citation_status=result.find_elements_by_tag_name('td')[12]
            data['citation_status'] = citation_status.text


            current_penalty=result.find_elements_by_tag_name('td')[13]
            data['current_penalty'] = current_penalty.text


            amount_paid=result.find_elements_by_tag_name('td')[14]
            data['amount_paid'] = amount_paid.text


            all_violations.append(data)   

        except:
            pass
    
    final_df = pd.DataFrame(all_violations)

    final_df.to_csv(str(row['id'])+"-mine.csv", index=False)

In [15]:
mines_df.apply(get_violations, axis=1)

sending Mines IDs
searching
Dirt Works
  
8778046  
000374480
10/14/2014
3/27/2015 
104(a)
10/14/2014
C
N
56.14132(a)
Dirt Works
  
8778046  
000374480
10/14/2014
3/27/2015 
104(a)
10/14/2014
C
N
56.14132(a)
Dirt Works
  
8778047  
000374480
10/14/2014
3/27/2015 
104(a)
11/4/2014
C
N
56.18010
Dirt Works
  
8771783  
000345454
1/22/2014
4/20/2014 
104(a)
1/22/2014
C
Y
56.9300(a)
Dirt Works
  
8771784  
000345454
1/22/2014
4/20/2014 
104(a)
1/22/2014
C
N
50.30(a)
Dirt Works
  
8771781  
000348280
1/22/2014
5/21/2014 
104(a)
1/22/2014
C
N
56.14100(b)
Dirt Works
  
8771782  
000345454
1/22/2014
4/20/2014 
104(a)
1/22/2014
C
Y
56.9300(a)
Dirt Works
  
8688404  
000316329
1/29/2013
4/19/2013 
104(a)
1/29/2013
C
N
50.30(a)
Dirt Works
  
8681997  
000316329
1/28/2013
4/19/2013 
104(a)
2/1/2013
C
N
47.31(a)
Dirt Works
  
8681998  
000316329
1/28/2013
4/19/2013 
104(a)
2/1/2013
C
Y
56.9300(a)
Dirt Works
  
8681995  
000316329
1/28/2013
4/19/2013 
104(a)
2/1/2013
C
N
56.14132(a)
Dirt Works
  
868

104(a)
5/6/2003
C
N
56.9300(a)
Holley Dirt Company, Inc
  
6092154  
080130605503
5/6/2003
7/25/2003 
104(a)
5/6/2003
C
N
56.20003(a)
Holley Dirt Company, Inc
  
7763979  
080130605502
11/13/2002
1/14/2003 
104(a)
11/13/2002
C
N
56.16006
Holley Dirt Company, Inc
  
7763976  
080130605502
11/13/2002
1/14/2003 
104(a)
11/14/2002
C
Y
56.14100(b)
Holley Dirt Company, Inc
  
7763977  
080130605502
11/13/2002
1/14/2003 
104(a)
11/13/2002
C
Y
56.14132(b)(1)(i)
Holley Dirt Company, Inc
  
7763978  
080130605502
11/13/2002
1/14/2003 
104(a)
11/13/2002
C
N
56.4230(a)(1)
Holley Dirt Company, Inc
  
7763975  
080130605502
11/13/2002
1/14/2003 
104(a)
11/13/2002
C
N
56.20003(a)
Holley Dirt Company, Inc
  
7764984  
080130605501
6/11/2002
10/18/2002 
104(a)
7/29/2002
C
N
62.130(a)
Holley Dirt Company, Inc
  
7764906  
080130605501
11/6/2001
10/18/2002 
107(a)
3/20/2002
O
N/A
 
Holley Dirt Company, Inc
  
7764904  
080130605501
11/6/2001
10/18/2002 
104(a)
1/9/2002
C
N
56.12028
Holley Dirt Company, I

0    None
1    None
2    None
dtype: object

# Okay, now do it for ALL of the mines

Open up `mines.csv` using pandas and do the same thing, it will just be for more mines this time.

In [16]:
mines_final = pd.read_csv('mines.csv', dtype={'id': str})

In [17]:
mines_final.apply(get_violations, axis=1)

sending Mines IDs
searching
Newberg Rock & Dirt
  
8992368  
000440809
5/2/2017
7/23/2017 
104(a)
5/2/2017
C
N
56.14107(a)
Newberg Rock & Dirt
  
8992368  
000440809
5/2/2017
7/23/2017 
104(a)
5/2/2017
C
N
56.14107(a)
Newberg Rock & Dirt
  
8992214  
000435717
2/7/2017
5/21/2017 
104(a)
2/8/2017
C
N
56.14107(a)
Newberg Rock & Dirt
  
8872462  
000398910
10/21/2015
1/13/2016 
104(a)
10/27/2015
C
N
56.14107(a)
Newberg Rock & Dirt
  
8872461  
000398910
10/21/2015
1/13/2016 
104(a)
10/27/2015
C
N
56.12004
Newberg Rock & Dirt
  
8790965  
000383840
4/15/2015
7/17/2015 
104(a)
4/15/2015
C
N
56.13015
Newberg Rock & Dirt
  
8790964  
000383840
4/13/2015
7/17/2015 
104(a)
4/15/2015
C
N
56.4101
Newberg Rock & Dirt
  
8790962  
000383840
4/10/2015
7/17/2015 
104(a)
4/10/2015
C
N
56.14107(a)
Newberg Rock & Dirt
  
8790963  
000383840
4/10/2015
7/17/2015 
104(a)
4/13/2015
C
Y
56.15020
Newberg Rock & Dirt
  
8694458  
000345066
1/29/2014
4/12/2014 
104(a)
1/29/2014
C
N
56.12018
Newberg Rock & Dirt


8/26/2004
C
Y
56.14107(a)
Dirt Company
  
6358295  
000041449
8/11/2004
12/23/2004 
104(a)
8/26/2004
C
Y
56.14130(a)(1)
Dirt Company
  
6358298  
000041449
8/11/2004
12/23/2004 
104(a)
8/15/2004
C
N
56.14107(a)
sending Mines IDs
searching
Sunshine Reclamation Inc.
  
8657165  
000291395
5/1/2012
7/25/2012 
104(a)
5/1/2012
C
Y
56.9300(b)
Sunshine Reclamation Inc.
  
8657165  
000291395
5/1/2012
7/25/2012 
104(a)
5/1/2012
C
Y
56.9300(b)
Sunshine Reclamation Inc.
  
8657164  
000291395
5/1/2012
7/25/2012 
104(a)
5/2/2012
C
N
56.14107(a)
Dirt Con
J392 
8580031  
000242289
10/27/2010
1/30/2011 
104(a)
10/27/2010
C
Y
56.14100(d)
Dirt Con
J392 
8580030  
000242289
10/27/2010
1/30/2011 
104(a)
10/27/2010
C
Y
56.14100(c)
Dirt Con
J392 
8580028  
000239419
10/26/2010
1/2/2011 
104(a)
10/26/2010
C
Y
56.14132(a)
Dirt Con
J392 
8580024  
000239419
10/26/2010
1/2/2011 
104(a)
10/27/2010
C
Y
56.14100(b)
Sunshine Reclamation Inc.
  
8580029  
000240535
10/26/2010
1/13/2011 
104(a)
10/26/2010
C
N
56.14

104(a)
10/6/1999
C
Y
56.12025
Sunshine Reclamation Inc.
  
7715665  
460825405505
11/18/1998
5/23/1999 
104(a)
1/11/1999
C
N
56.12028
Sunshine Reclamation Inc.
  
7715666  
460825405504
11/18/1998
3/28/1999 
104(a)
11/24/1998
C
Y
56.14107(a)
Sunshine Reclamation Inc.
  
7715667  
460825405504
11/18/1998
3/28/1999 
104(a)
11/18/1998
C
Y
56.12025
R & J Contracting
WGP 
4439826  
460825405501WGP
9/28/1995
5/4/1996 
104(a)
10/13/1995
C
N
56.14100(c)
sending Mines IDs
searching
Dirt Doctor Inc
  
8946233  
000422613
8/31/2016
11/25/2016 
104(a)
8/31/2016
C
Y
56.18002(a)
Dirt Doctor Inc
  
8946233  
000422613
8/31/2016
11/25/2016 
104(a)
8/31/2016
C
Y
56.18002(a)
Dirt Doctor Inc
  
8946234  
000422613
8/31/2016
11/25/2016 
104(a)
9/13/2016
C
N
56.14132(a)
Dirt Doctor Inc
  
8946232  
000422613
8/31/2016
11/25/2016 
104(a)
9/7/2016
C
N
56.12018
Dirt Doctor Inc
  
8946226  
000422613
8/24/2016
11/25/2016 
104(a)
8/24/2016
C
Y
56.14100(c)
Dirt Doctor Inc
  
8946218  
000422613
8/24/2016
11/25/2

Holley Dirt Company, Inc
  
8547866  
000230512
7/14/2010
10/9/2010 
104(a)
7/15/2010
C
Y
56.9300(a)
Holley Dirt Company, Inc
  
8547863  
000230512
7/14/2010
10/9/2010 
104(a)
7/15/2010
C
N
56.14107(a)
Holley Dirt Company, Inc
  
8547862  
000230512
7/14/2010
10/9/2010 
104(a)
7/15/2010
C
N
56.15020
Holley Dirt Company, Inc
  
8544318  
000218132
3/19/2010
6/4/2010 
104(a)
3/19/2010
C
N
56.14132(a)
Holley Dirt Company, Inc
  
8544317  
000218132
3/19/2010
6/4/2010 
104(a)
3/19/2010
C
N
56.12004
Holley Dirt Company, Inc
  
7751779  
000158499
6/19/2008
9/5/2008 
104(a)
7/8/2008
C
N
56.14103(a)
Holley Dirt Company, Inc
  
7751780  
000158499
6/19/2008
9/5/2008 
104(d)(1)
6/20/2008
C
Y
56.3131
Holley Dirt Company, Inc
  
7780184  
000141737
1/8/2008
4/2/2008 
104(a)
1/9/2008
C
N
56.12008
Holley Dirt Company, Inc
  
7780185  
000141737
1/8/2008
4/2/2008 
104(a)
1/9/2008
C
N
56.14112(b)
Holley Dirt Company, Inc
  
7780187  
000141737
1/8/2008
4/2/2008 
104(a)
1/9/2008
C
N
56.12018
Holley D

C
N
56.14130(c)
M R Dirt
  
6060560  
000158367
5/12/2008
9/3/2008 
104(a)
5/15/2008
C
N
56.20008(a)
M R Dirt
  
6060556  
000158367
5/12/2008
9/3/2008 
104(a)
5/12/2008
C
N
56.1000
M R Dirt
  
6060562  
000158367
5/12/2008
9/3/2008 
104(a)
6/17/2008
C
N
56.18010
M R Dirt
  
6060557  
000158367
5/12/2008
9/3/2008 
104(a)
6/3/2008
C
N
46.3(a)
M R Dirt
  
6060558  
000158367
5/12/2008
9/3/2008 
104(a)
6/3/2008
C
N
56.14132(a)
M R Dirt
  
6060561  
000158367
5/12/2008
9/3/2008 
104(a)
5/15/2008
C
N
56.4200(a)(1)
M R Dirt
  
6060555  
000158367
5/12/2008
9/3/2008 
104(a)
5/15/2008
C
N
47.31(a)
sending Mines IDs
searching
M.R. Dirt Inc.
  
9317668  
000442816
5/11/2017
8/19/2017 
104(a)
5/11/2017
C
N
50.30(a)
M.R. Dirt Inc.
  
9317668  
000442816
5/11/2017
8/19/2017 
104(a)
5/11/2017
C
N
50.30(a)
M.R. Dirt Inc.
  
8928850  
000421654
8/8/2016
11/10/2016 
104(a)
8/8/2016
C
N
56.9301
M.R. Dirt Inc.
  
8807882  
000380669
4/2/2015
6/7/2015 
104(a)
4/2/2015
C
N
56.14132(a)
M.R. Dirt Inc.
  
865

4/30/2013
C
N
56.20003(a)
R D Blankenship Dirt Work LLC
  
8688887  
000324075
4/29/2013
7/13/2013 
104(a)
4/30/2013
C
N
56.12002
R D Blankenship Dirt Work LLC
  
8688892  
000324075
4/29/2013
7/13/2013 
104(a)
4/30/2013
C
N
56.12004
R D Blankenship Dirt Work LLC
  
8688890  
000324075
4/29/2013
7/13/2013 
104(a)
5/8/2013
C
N
56.12002
R D Blankenship Dirt Work LLC
  
8688889  
000324075
4/29/2013
7/13/2013 
104(a)
4/30/2013
C
Y
56.14109
R D Blankenship Dirt Work LLC
  
8688886  
000324075
4/29/2013
7/13/2013 
104(a)
5/8/2013
C
N
56.12004
R D Blankenship Dirt Work LLC
  
8685443  
000315892
1/29/2013
4/14/2013 
104(a)
1/29/2013
C
N
56.12004
R D Blankenship Dirt Work LLC
  
8685442  
000315892
1/29/2013
4/14/2013 
104(a)
1/29/2013
C
N
56.14132(a)
R D Blankenship Dirt Work LLC
  
8616682  
000260355
5/10/2011
8/12/2011 
104(a)
5/10/2011
C
N
56.20003(a)
R D Blankenship Dirt Work LLC
  
8616681  
000260355
5/10/2011
8/12/2011 
104(a)
5/11/2011
C
N
56.12034
R D Blankenship Dirt Work LLC
  
8

5/8/2013
C
N
56.20003(a)
Spry's Dirt & Gravel, Inc.
  
8681533  
000325641
5/8/2013
8/9/2013 
104(a)
5/8/2013
C
N
56.14107(a)
Spry's Dirt & Gravel, Inc.
  
8676608  
000287903
3/20/2012
6/7/2012 
104(a)
3/20/2012
C
N
56.9300(b)
Spry's Dirt & Gravel, Inc.
  
8615969  
000270764
9/20/2011
12/8/2011 
104(a)
9/21/2011
C
N
56.14132(a)
Spry's Dirt & Gravel, Inc.
  
8615968  
000270764
9/20/2011
12/8/2011 
104(a)
9/20/2011
C
N
56.14112(a)(1)
Spry's Dirt & Gravel, Inc.
  
8615967  
000270764
9/20/2011
12/8/2011 
104(a)
9/20/2011
C
Y
56.12005
Spry's Dirt & Gravel, Inc.
  
6574225  
000224303
5/10/2010
8/5/2010 
104(a)
5/10/2010
C
N
50.30(a)
Spry's Dirt & Gravel, Inc.
  
6574206  
000220879
3/30/2010
7/9/2010 
104(a)
4/1/2010
C
Y
56.11001
Spry's Dirt & Gravel, Inc.
  
6574207  
000220879
3/30/2010
7/9/2010 
104(a)
4/1/2010
C
N
56.12030
Spry's Dirt & Gravel, Inc.
  
6574208  
000220879
3/30/2010
7/9/2010 
104(a)
4/1/2010
C
Y
56.14107(a)
Spry's Dirt & Gravel, Inc.
  
6467865  
000183651
3/25/2009


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object